In [267]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
df = pd.read_csv('https://query.data.world/s/5xzuftmozqteqbqzopbcgxbjzusyi7')

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...
...,...,...,...,...
39995,1753918954,neutral,showMe_Heaven,@JohnLloydTaylor
39996,1753919001,love,drapeaux,Happy Mothers Day All my love
39997,1753919005,love,JenniRox,Happy Mother's Day to all the mommies out ther...
39998,1753919043,happiness,ipdaman1,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [210]:
#split the training set and test set with test_size = 0.4
def split_train_test(df):
    train_set, test_set = train_test_split(df, test_size=0.4, random_state=42)

In [211]:
#reomve pattern function, input_text is the text we want to process, the pattern is the pattern we want to remove
def remove_pattern(input_text, pattern):
    r = re.findall(pattern, input_text)
    for i in r:
        input_text = re.sub(i, '', input_text)
    return input_text

In [212]:
emoji = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)

In [213]:
#preprocess the data
def preprocess(df):
    split_train_test(df)
    #combine the train_set and test_set to preprocess the data
    combi = pd.concat([train_set,test_set], ignore_index=True)
    
    #remove the @
    combi['processed'] = np.vectorize(remove_pattern)(combi['content'], "@[\w]*")
    
    #remove the #
    combi['processed'] = np.vectorize(remove_pattern)(combi['processed'], "#")
    
    #remove emoji
    combi['processed'] = np.vectorize(remove_pattern)(combi['processed'], emoji)
    
    #remove the special characters, numbers, punctuations
    combi['processed'] = combi['processed'].str.replace("[^a-zA-Z#]", " ",regex=True)
    
     #remove the words smaller than 2
    combi['processed'] = combi['processed'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
    
    #drop sentiment
    drop_index = combi[(combi['sentiment']=='empty') | (combi['sentiment']=='enthusiasm') | (combi['sentiment']=='fun')].index
    combi = combi.drop(drop_index)
    return combi

In [214]:
df = pd.read_csv('https://query.data.world/s/5xzuftmozqteqbqzopbcgxbjzusyi7')

In [231]:
df_processed = preprocess(df)
df_processed

,tweet_id,sentiment,author,content,processed
0,1694088996,love,whyamievenhere,@zoeatthedisco lol hell yes i'm keen. WE'RE GO...,lol hell yes keen GOING SKIING TREBLE CONE SOM...
1,1751715234,love,theamericanxp,Feeling special @ looking4him first guy to giv...,Feeling special looking him first guy give flo...
3,1694272881,neutral,akane_takamura,?and make your own pledge while you're at it!,and make your own pledge while you
4,1753094560,worry,CVJason,@mosdefaqueen My pleasure. I can't be with my ...,pleasure can with mom Mother Day But can sprea...
5,1961438666,worry,santoleto,"6:29 pm - ok, let's go now through #bowman #st...",let now through bowman strategicClock but firs...
...,...,...,...,...,...
39995,1957212416,neutral,summatusmentis,"really, realy want a netbook. mrr. Techno-lust...",really realy want netbook mrr Techno lust sucks
39996,1751921888,happiness,GaByDiAz,@ Butlers watching Dr. Farmer rock out w/ the ...,Butlers watching Farmer rock out the hispanic ...
39997,1695220613,surprise,jesssicababesss,"@iyaitssuzanne ohh yeh , but he was on sexy me...",ohh yeh but was sexy men its okay
39998,1963521553,worry,jerrybruno,Just found about ten typos in one story in the...,Just found about ten typos one story the Plain...


In [256]:
senti_list = df_processed['sentiment'].unique().tolist()

In [281]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\11580\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [301]:
text = []
freq_text = []
stop = set(stopwords.words('english'))
import nltk
from sklearn.feature_extraction.text import CountVectorizer
for str in df_processed[(df_processed['sentiment']=='love')]['processed']:
    freq_text.extend([x.lower() for x in str.split(' ') if x not in stop])
    text.append(str.lower())

cv = CountVectorizer(max_df=0.9, min_df=2, stop_words='english')
cv.fit_transform(text)
print(freq_text)
from nltk import FreqDist
fdist = FreqDist(freq_text)
sorted(fdist.items(), key=lambda kv: kv[1], reverse=True)

['lol', 'hell', 'yes', 'keen', 'going', 'skiing', 'treble', 'cone', 'sometime', 'this', 'winter', 'feeling', 'special', 'looking', 'first', 'guy', 'give', 'flowers', 'home', 'dinner', 'mum', 'delicious', 'nice', 'good', 'time', 'happy', 'mothers', 'day', 'mums', 'agree', 'mmm', 'good', 'miss', 'eating', 'tho', 'know', 'loved', 'wolves', 'wall', 'totally', 'awesome', 'tip', 'work', 'tiny', 'format', 'got', 'inspiration', 'luv', 'movie', 'say', 'anything', 'http', 'blip', 'yroy', 'what', 'compiment', 'glad', 'good', 'time', 'and', 'thanks', 'gorgeous', 'flowers', 'let', 'make', 'last', 'forever', 'kickin', 'bak', 'need', 'miss', 'babe', 'lolz', 'really', 'wants', 'see', 'tonight', 'mmmmmm', 'mcdonalds', 'coffee', 'soooooo', 'good', 'maybe', 'going', 'hookah', 'tonight', 'abbster', 'unhooking', 'home', 'sleep', 'sleep', 'good', 'thatd', 'likely', 'bein', 'lived', 'campus', 'moved', 'back', 'cov', 'http', 'twtvite', 'com', 'koyqo', 'twtvite', 'aptw', 'have', 'miss', 'engagements', 'aus', '

[('love', 865),
 ('day', 791),
 ('happy', 601),
 ('good', 325),
 ('mothers', 303),
 ('mother', 302),
 ('http', 223),
 ('quot', 208),
 ('thanks', 196),
 ('like', 187),
 ('com', 153),
 ('mom', 152),
 ('lol', 149),
 ('great', 148),
 ('you', 141),
 ('really', 136),
 ('amp', 136),
 ('time', 133),
 ('see', 132),
 ('today', 132),
 ('much', 128),
 ('one', 128),
 ('new', 122),
 ('get', 121),
 ('got', 116),
 ('night', 113),
 ('hope', 107),
 ('thank', 106),
 ('best', 106),
 ('haha', 104),
 ('know', 100),
 ('miss', 98),
 ('awesome', 97),
 ('the', 96),
 ('going', 94),
 ('nice', 94),
 ('morning', 88),
 ('amazing', 87),
 ('want', 85),
 ('back', 84),
 ('well', 83),
 ('home', 81),
 ('twitpic', 80),
 ('lovely', 78),
 ('fun', 78),
 ('cute', 76),
 ('wish', 74),
 ('moms', 73),
 ('loved', 72),
 ('work', 71),
 ('think', 70),
 ('better', 67),
 ('still', 66),
 ('tomorrow', 66),
 ('just', 66),
 ('would', 64),
 ('guys', 62),
 ('twitter', 62),
 ('last', 61),
 ('sweet', 61),
 ('tonight', 60),
 ('say', 59),
 ('and'